# What's in an Avocado Toast: A Supply Chain Analysis

![](avocado_wallpaper.jpeg)

You find yourself in London, crafting a delectable avocado toast, a dish that has risen dramatically in popularity on breakfast menus since the 2010s. This straightforward recipe requires just five ingredients: a ripe avocado, half a lemon, a generous pinch of salt flakes, two slices of sourdough bread, and a good drizzle of extra virgin olive oil. Most of these ingredients are now staples in grocery stores, and as you will find with this project, that is no small feat!

In this project, you'll conduct a supply chain analysis of three ingredients used in avocado toast using the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin.
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple category tags in the `categories_tags` column.
There is a column in each CSV file called `origins_tags`, which contains strings for the country of origin of each item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

[Open Food Facts database](https://world.openfoodfacts.org/)

In [252]:
import pandas as pd

In [ ]:
def get_top(item=str('avocado')):
    path_item = 'data/' + item + '.csv'
    df_item = pd.read_csv(path_item,sep='\t')

    subset_columns = [ 'code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']
    df_item = df_item[subset_columns]
    df_item = df_item.dropna(subset=['categories_tags'])

    df_item['categories_tags'] = df_item.categories_tags.str.split(',')

    path_categories = 'data/relevant_' + item + '_categories.txt'
    categories_item = pd.read_csv(path_categories, header = None)
    categories_item.columns = ['categories_tags']

    mask_categories = []
    for index,tags in enumerate(df_item['categories_tags'].values.tolist()):
        for tag in tags:
            if tag in categories_item['categories_tags'].values.tolist():
                mask_categories.append(index)
                break
    df_item = df_item.iloc[mask_categories,:]

    item_uk = df_item[df_item['countries']=='United Kingdom']

    top_item = item_uk.value_counts('origins_tags').reset_index()
    top_item.origins_tags = top_item.origins_tags.str.split(',')
    top_item = top_item.explode('origins_tags')

    top_item.columns = ['origins_tags', 'count']
    top_item = top_item.groupby('origins_tags').sum().reset_index().sort_values(by=['count'],ascending=False)
    
    top_item_origin = top_item.iloc[0,:]['origins_tags']
    
    print('Top origin for ' + item + ': ' + top_item.iloc[0,:]['origins_tags'][3:] + ' with ' + str(top_item.iloc[0,:]['count']) + ' products')
    
    return top_item_origin[3:]

In [256]:
top_avocado_origin  = get_top('avocado')
top_olive_oil_origin = get_top('olive_oil')
top_sourdough_origin = get_top('sourdough')

Top origin for avocado: peru with 4 products


C:\Users\franf\AppData\Local\Temp\ipykernel_13040\1863735670.py:3: DtypeWarning: Columns (0,1,2,4,5,6,7,8,10,13,15,16,17,18,20,21,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,38,39,42,43,44,45,46,48,49,50,57,67,70,71,72,73,74,75,76,79,80,81,82,83,86,87,88,89,90,91,92,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,115,121,125,129,133,137,141,145,149,153,157,159,161,165,167,169,177,179,181,183,185,187,189,191,193,195,197,199,201,203,205,211,213,215,217,219,221,223,225,227,229,231,233,235,237,239,241,243,245,247,249,251,253,255,257,259,261,263,265,267,269,271,273,274,275,276,277,278,279,280,281,282,283,284,285,286,288,289,290,291,292,293,294,305,311,312,313,315,316,319,320,321,322,324) have mixed types. Specify dtype option on import or set low_memory=False.
  df_item = pd.read_csv(path_item,sep='\t')


Top origin for olive_oil: greece with 8 products
Top origin for sourdough: united-kingdom with 3 products
